# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

¡Empecemos!

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=20&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=20/12/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='
r = requests.get(url)

In [ ]:
r.status_code

In [ ]:
s = BeautifulSoup(r.text, 'lxml')
print(s.prettify())

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

In [ ]:
with open('latam.html', 'w') as f:
    f.write(r.text)
    f.close

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [ ]:
from selenium import webdriver

Paso 1: instanciar un **driver** del navegador

In [ ]:
driver = webdriver.Chrome(executable_path='../chromedriver')

Paso 2: hacer que el navegador cargue la página web.

In [ ]:
driver.get(url)

Paso 3: extraer la información de la página

In [ ]:
# 

Paso 4: cerrar el navegador

In [ ]:
driver.close()

In [ ]:
# Podemos agregarle opciones al driver para utilizar los distintos modos del Chrome
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../../chromedriver', options=options)
driver.get(url)

cada vuelo es un `<li class='flight'>` 

In [ ]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')

Nos quedamos con un vuelo para analizarlo

In [ ]:
vuelo = vuelos[0]

#### Hora de llegada, hora de salida y duración

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

#### Escalas

Debemos clickear el botón de las escalas para conocer esa información:los números de vuelo, la duración de los vuelos y la duración de las escalas

In [ ]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
boton_escalas

In [ ]:
boton_escalas.click()

In [ ]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
segmentos

In [ ]:
escalas = len(segmentos) - 1 #0 escalas si es un vuelo directo

Me quedo con un segmento para analizarlo

In [ ]:
segmento = segmentos[0]

In [ ]:
segmento.get_attribute('class')

In [ ]:
segmento.text

De cada segmento necesitamos:
- origen
- hora de salida
- destino
- hora de llegada
- duración del vuelo
- numero de vuelo
- modelo de avion
- duración de la escala

In [ ]:
# Origen
segmento.find_element_by_xpath('.//div[@class="departure"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
# Destino
segmento.find_element_by_xpath('.//div[@class="arrival"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de llegada
segmento.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Duración del vuelo
segmento.find_element_by_xpath('.//span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')

In [ ]:
# Numero del vuelo
segmento.find_element_by_xpath('.//span[@class="equipment-airline-number"]').text

In [ ]:
# Modelo de avion
segmento.find_element_by_xpath('.//span[@class="equipment-airline-material"]').text

In [ ]:
# Duracion de la escala
segmento.find_element_by_xpath('.//div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')

Juntamos todo en una función que itere en cada segmento

In [ ]:
info_escalas = []
for segmento in segmentos:
    # Origen
    origen = segmento.find_element_by_xpath(
        './/div[@class="departure"]/span[@class="ground-point-name"]').text
    # Hora de salida
    dep_time = segmento.find_element_by_xpath(
        './/div[@class="departure"]/time').get_attribute('datetime')
    # Destino
    destino = segmento.find_element_by_xpath(
        './/div[@class="arrival"]/span[@class="ground-point-name"]').text
    # Hora de llegada
    arr_time = segmento.find_element_by_xpath(
        './/div[@class="arrival"]/time').get_attribute('datetime')
    # Duración del vuelo
    duracion_vuelo = segmento.find_element_by_xpath(
        './/span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
    # Numero del vuelo
    numero_vuelo = segmento.find_element_by_xpath(
        './/span[@class="equipment-airline-number"]').text
    # Modelo de avion
    modelo_avion = segmento.find_element_by_xpath(
        './/span[@class="equipment-airline-material"]').text
    # Duracion de la escala
    if segmento != segmentos[-1]:
        duracion_escala = segmento.find_element_by_xpath(
            './/div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
    else:
        duracion_escala = ''
        
    # Armo un diccionario para almacenar los datos
    data_dict={'origen': origen, 
                'dep_time': dep_time, 
                'destino': destino,
                'arr_time': arr_time,
                'duracion_vuelo': duracion_vuelo,
                'numero_vuelo': numero_vuelo,
                'modelo_avion': modelo_avion,
                'duracion_escala': duracion_escala}
    print(data_dict)
    info_escalas.append(data_dict)

Debemos cerrar el pop-up 

In [ ]:
driver.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()

#### Precios disponibles

In [ ]:
vuelo.click()

La información de los precios para cada tarifa está contenida en una tabla. Los precios en sí están en el footer y podemos sacar los nombres de la clase de cada elemento

In [ ]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [ ]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

## Juntamos todo

In [ ]:
def obtener_precios(vuelo):
    tarifas = vuelo.find_elements_by_xpath(
        './/div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
        dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
    return precios

In [ ]:
def obtener_datos_escalas(vuelo):
    segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
    info_escalas = []
    for segmento in segmentos:
        # Origen
        origen = segmento.find_element_by_xpath(
            './/div[@class="departure"]/span[@class="ground-point-name"]').text
        # Hora de salida
        dep_time = segmento.find_element_by_xpath(
            './/div[@class="departure"]/time').get_attribute('datetime')
        # Destino
        destino = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/span[@class="ground-point-name"]').text
        # Hora de llegada
        arr_time = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/time').get_attribute('datetime')
        # Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath(
            './/span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
        # Numero del vuelo
        numero_vuelo = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-number"]').text
        # Modelo de avion
        modelo_avion = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-material"]').text
        # Duracion de la escala
        if segmento != segmentos[-1]:
            duracion_escala = segmento.find_element_by_xpath(
                './/div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
        else:
            duracion_escala = ''

        # Armo un diccionario para almacenar los datos
        data_dict={'origen': origen, 
                    'dep_time': dep_time, 
                    'destino': destino,
                    'arr_time': arr_time,
                    'duracion_vuelo': duracion_vuelo,
                    'numero_vuelo': numero_vuelo,
                    'modelo_avion': modelo_avion,
                    'duracion_escala': duracion_escala}
        info_escalas.append(data_dict)
        
    return info_escalas

In [ ]:
def obtener_tiempos(vuelo):
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    # Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    return {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}

In [ ]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('Iniciando scraping...')
    info = []
    for vuelo in vuelos:
        # Obtenemos los tiempos generales del vuelo
        tiempos = obtener_tiempos(vuelo)
        # Clickeamos el botón de escalas para ver los detalles
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button').click()
        escalas = obtener_datos_escalas(vuelo)
        # Cerramos el pop-up con los detalles
        vuelo.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()
        # Clickeamos el vuelo para ver los precios
        vuelo.click()
        precios = obtener_precios(vuelo)
        # Cerramos los precios del vuelo
        vuelo.click()
        info.append({'precios':precios, 'tiempos':tiempos , 'escalas': escalas})
    return info

Agregamos funcionalidades para que espere a que cargue la página

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
url='https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=25&fecha1_anomes=2019-10&fecha2_dia=12&fecha2_anomes=2019-11&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=25/10/2019&vuelos_fecha_regreso_ddmmaaaa=12/11/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../../chromedriver', options=options)
driver.get(url)
delay = 10
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print("La página terminó de cargar")
    info = obtener_info(driver)
except TimeoutException:
    print("La página tardó demasiado en cargar")
driver.close()

Ahora debemos ver cómo construir la URL del sitio para hacer consultas

In [ ]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=20&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=20/12/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='

In [ ]:
url_base = 'https://www.latam.com/es_ar/apps/personas/booking?' 

In [ ]:
params = url.strip(url_base).split('&')
params

In [ ]:
import time

In [ ]:
fecha = '10/12/2019'

In [ ]:
fecha = time.strptime(fecha, '%d/%m/%Y')

In [ ]:
time.strftime('%d/%m/%Y',fecha)

In [ ]:
def armar_url(url_base, fecha, origen, destino, cabina):
    url = url_base
    url+=f'&fecha1_dia={fecha.tm_mday}'
    url+=f'&fecha1_anomes={fecha.tm_year}-{fecha.tm_mon}'
    url+=f'&auAvailability=1'
    url+=f'&ida_vuelta=ida'
    url+=f'&from_city1={origen}'
    url+=f'&to_city1={destino}'
    url+=f'&vuelos_fecha_salida_ddmmaaaa={time.strftime("%d/%m/%Y",fecha)}'
    url+=f'&cabina={cabina}'
    url+=f'&nadults=1'
    return url

In [ ]:
url = armar_url(url_base,fecha, 'MAD', 'BUE', 'Y')
url

In [ ]:
def scrape_latam(urls):
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
    delay = 10
    # Si es un string único, lo convierto en lista
    if type(urls) == str:
        urls = [urls]

    print(urls)
    info = []
    for url in urls:
        print('Scraping URL:',url)
        driver.get(url)
        try:
            myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
            print("Page is ready!")
            info.append(obtener_info(driver))
        except TimeoutException:
            print("Loading took too much time!")
    driver.close()
    return info

In [ ]:
scrape_latam(url)